<a href="https://colab.research.google.com/github/Dheeraj024/MachineSVM/blob/main/ML_SVM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import svm
import os,time
from sklearn.multiclass import  OneVsRestClassifier
from sklearn.metrics import accuracy_score


In [5]:
hyp_data = sio.loadmat("/content/sample_data/Indian_pines_corrected.mat")
gt_data = sio.loadmat("/content/sample_data/Indian_pines_gt.mat")

In [6]:
data = hyp_data[sorted(hyp_data.keys())[-1]]

def applyPCA(X, n_components=30, seed=1):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=n_components, whiten=True, random_state=seed)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0], X.shape[1], n_components))
    return newX

In [7]:
data.shape


(145, 145, 200)

In [8]:
data = applyPCA(data)
data.shape

labels = gt_data[sorted(gt_data.keys())[-1]]

print(data.shape)
print(labels.shape)

data = data.reshape(data.shape[0]*data.shape[1],data.shape[2])
data.shape

labels = labels.reshape(-1,1)
labels.shape

(145, 145, 30)
(145, 145)


(21025, 1)

In [9]:
data.shape

(21025, 30)

In [10]:
combine = np.concatenate((data,labels),axis=1)
combine

array([[ 0.96876849,  0.4801512 ,  0.09501285, ..., -0.08308338,
        -0.3144414 ,  3.        ],
       [ 1.08206297, -0.66688618,  0.45761543, ..., -1.70082187,
         3.01117831,  3.        ],
       [ 1.11968468, -1.01852852,  0.64112087, ...,  0.83300539,
        -0.37002131,  3.        ],
       ...,
       [-1.42368833, -0.37320899,  0.52039646, ...,  0.37675531,
         1.08857678,  0.        ],
       [-1.255093  ,  0.26308297,  0.88456889, ...,  0.01608009,
        -0.01004176,  0.        ],
       [-1.26926721,  0.11428131,  0.60833014, ...,  0.53927323,
         0.54789815,  0.        ]])

In [11]:
df = pd.DataFrame(combine)

In [12]:
df

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.968768,0.480151,0.095013,0.125597,-0.835529,-0.153065,0.340914,-0.808389,0.739768,-0.196726,...,-0.680243,-0.742375,0.309232,0.706960,-0.505994,0.346319,-0.390967,-0.083083,-0.314441,3.0
1,1.082063,-0.666886,0.457615,-0.932147,0.283993,-0.641161,0.513249,0.639667,-1.369898,0.276044,...,-0.074711,0.634126,0.264466,-0.794392,-0.303542,1.126113,-0.518024,-1.700822,3.011178,3.0
2,1.119685,-1.018529,0.641121,-1.340924,0.498481,-0.292635,0.437308,0.546881,1.651552,-0.565767,...,0.060651,0.936132,0.691308,-1.013655,-0.927224,0.821306,0.184328,0.833005,-0.370021,3.0
3,1.079131,-0.780896,0.465640,-0.886678,0.281052,-0.681895,0.634920,0.624044,-0.905912,0.162279,...,0.120929,0.673786,0.522048,-0.722938,-0.164347,1.201609,-0.673626,-1.598063,2.643712,3.0
4,0.969944,0.111926,-0.030070,-0.163259,-0.706674,-0.974477,1.271296,-0.917720,-0.543372,-0.199309,...,-0.232870,-0.182736,0.318033,0.060125,-0.217072,0.644748,-0.392546,0.256941,0.084382,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21020,-1.446429,-0.595113,0.362794,0.393763,-0.140957,-0.115103,0.776805,-0.823793,-1.194199,0.297644,...,0.431289,-0.711490,1.542814,-0.722164,1.513280,-0.486355,1.319352,-0.204216,-0.042527,0.0
21021,-1.496650,-0.603758,0.522121,0.160608,-0.764288,0.261460,0.927415,-0.550354,-0.718704,0.020032,...,1.871089,-2.438211,2.623816,-1.649280,0.842920,0.543669,2.358727,0.030367,0.068563,0.0
21022,-1.423688,-0.373209,0.520396,0.166038,-0.689525,0.233610,0.910813,-0.402465,0.425074,0.019419,...,0.913735,-0.124981,0.628442,-0.339900,1.258925,-0.756236,0.650048,0.376755,1.088577,0.0
21023,-1.255093,0.263083,0.884569,-0.347298,-1.470756,1.006945,0.644286,0.788673,0.618766,0.047805,...,1.553205,0.001168,-0.410227,0.505436,-1.213134,-0.077920,-0.151054,0.016080,-0.010042,0.0


In [13]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

In [14]:
C_values = [0.1, 1, 10, 100]
kernel_type = ['linear','poly','rbf']
accuracy = []

In [ ]:
for kernel in kernel_type:
  for C in C_values:
      # Initialize SVM classifier with current C value
      svm = SVC(kernel=kernel, C=C)

      start_time1 = time.time()
      svm_ovr = OneVsRestClassifier(svm).fit(X_train, y_train)
      Time_taken_ovr = time.time() - start_time1
      y_test_ovr = svm_ovr.predict(X_test)

      acc_ovr = accuracy_score(y_test, y_test_ovr)
      accuracy.append(acc_ovr)

      
      # # Train SVM classifier
      # svm.fit(X_train, y_train)
      
      # # Get number of support vectors
      # n_support_vectors = svm.n_support_
      
      # # Print number of support vectors for current C value
      # print(f"C = {C}: Number of support vectors = {sum(n_support_vectors)}")